In [1]:
import os
import sys

current_dir = os.getcwd()

parent_dir = os.path.abspath(os.path.join(current_dir, ".."))

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [2]:
import configargparse

parser = configargparse.ArgParser(default_config_files=['../configs/config.ini'])

parser.add('--config', is_config_file=True, help='Path to config file')
parser.add('--data-folder', type=str, required=True, help='Root data directory containing Images and CHM folders')
parser.add('--output-dir', type=str, default='output/chmpredict', help='Directory to save output models and logs')
parser.add('--learning-rate', type=float, default=1e-4, help='Learning rate for optimizer')
parser.add('--batch-size', type=int, default=16, help='Batch size for DataLoader')
parser.add('--epochs', type=int, default=50, help='Number of training epochs')
parser.add('--patience', type=int, default=5, help='Patience for early stopping')

config, _ = parser.parse_known_args()

print(config)

Namespace(config=None, data_folder='/Users/anisr/Documents/Finland_CHM', output_dir='output/chmpredict', learning_rate=0.0001, batch_size=16, epochs=50, patience=5)


In [3]:
import torch

from chmpredict.data.loader import load_fn
from chmpredict.model.build import build_fn
from chmpredict.model.train import train_fn
from chmpredict.model.eval import eval_fn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if not os.path.exists(config.output_dir):
    os.makedirs(config.output_dir)

train_loader, val_loader, test_loader = load_fn(config.data_folder, config.batch_size)

model, criterion, optimizer = build_fn(config.learning_rate, device)

train_fn(train_loader, val_loader, model, criterion, optimizer, config.epochs, config.patience, config.output_dir, device)

eval_fn(test_loader, model, criterion, config.output_dir, device)

RuntimeError: Parent directory output/chmpredict does not exist.